## Import Libraries

In [1]:
import gym
from gym import wrappers
from IPython.display import HTML
import glob
import os
import random
import numpy as np
import tensorflow as tf 
import tensorflow.contrib.slim as slim
from collections import deque
from numpy.random import normal, multivariate_normal

os.chdir("/home/devin/notebooks/pendulum-ddpg")

## DDPG Agent

In [2]:
# class DDPGAgent:
#     def __init__(self, state_size, action_size):
#         self.state_size = state_size
#         self.action_size = action_size
#         self.memory = deque(maxlen = 10000)
#         self.gamma = 0.95
#         self.tau = 0.125
#         self.learning_rate = 0.0001
#         state_input_ph = tf.placeholder(tf.float32, shape = (None, ob_len))
#         action_input_ph = tf.placeholder(tf.float32, shape = (None, action_len))
#         target_q_ph = tf.placeholder(tf.float32, shape = (None, 1))
        
#         with tf.variable_scope("actor"):
#             actor_output = self.build_actor(state_input_ph)

#         with tf.variable_scope("critic"):
#             critic_output = self.build_critic(state_input_ph, action_input_ph)

#         with tf.variable_scope("target_actor"):
#             target_actor_output = self.build_actor(state_input_ph)

#         with tf.variable_scope("target_critic"):
#             target_critic_output = self.build_critic(state_input_ph, actor_output)
            
#         actor_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'actor')
#         critic_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'critic')
#         target_actor_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'target_actor')
#         target_critic_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope = 'target_critic')
        
#         update_target_ops = []
#         for i in range(len(actor_weights)):
#             update_target_op = target_actor_weights[i].assign(tau*actor_weights[i] + (1-tau)*target_actor_weights[i])
#             update_target_ops.append(update_target_op)
#         for i in range(len(critic_weights)):
#             update_target_op = target_critic_weights[i].assign(tau*critic_weights[i] + (1-tau)*target_critic_weights[i])
#             update_target_ops.append(update_target_op)

#         critic_lose = tf.reduce_mean(tf.square(target_q_ph-critic_output))
#         critic_optimizer = tf.train.AdamOptimizer(learn_rate).minimize(critic_lose, var_list=critic_weights)

#         actor_lose = tf.reduce_mean(-target_critic_output)
#         actor_optimizer = tf.train.AdamOptimizer(learn_rate).minimize(actor_lose, var_list=actor_weights)

#     def build_actor(self, state_input):
#         actor_fc_1 = slim.fully_connected(state_input, 100, activation_fn = tf.nn.relu)
#         actor_fc_2 = slim.fully_connected(actor_fc_1, 50, activation_fn = tf.nn.relu)
#         actor_fc_3 = slim.fully_connected(actor_fc_2, 25, activation_fn = tf.nn.relu)
#         actor_output = slim.fully_connected(actor_fc_3, self.action_size, activation_fn = tf.nn.tanh) * 2
#         return actor_output
    
#     def build_critic(self, state_input, action_input):
#         critic_input = slim.flatten(tf.concat([state_input, action_input], axis = 1))
#         critic_fc_1 = slim.fully_connected(critic_input, 100, activation_fn = tf.nn.relu)
#         critic_fc_2 = slim.fully_connected(critic_fc_1, 50, activation_fn = tf.nn.relu)
#         critic_fc_3 = slim.fully_connected(critic_fc_2, 25, activation_fn = tf.nn.tanh)
#         critic_output = slim.fully_connected(critic_fc_3, self.action_size, activation_fn = None)
#         return critic_output
    
#     def act(self, state):
#         state = np.reshape(np.array(state), (-1, 3))
#         action = sess.run(actor_output, {state_input_ph: state})
#         return action
    
    

In [3]:
gym_env = 'Pendulum-v0'
env = gym.make(gym_env)
# print(env.observation_space)
# print(env.observation_space.low)
# print(env.observation_space.high)
# print(env.action_space)
# print(env.action_space.low)
# print(env.action_space.high)
# obs = env.reset()
# print(obs)
ob_len = len(env.observation_space.low)
action_len = len(env.action_space.low)

In [4]:
discount_factor = 0.9
hidden_size = 16
tau = 0.05
learn_rate = 1e-3

replay_memory = deque(maxlen=1000000)

def sample_from_memory(batch_size):
    return random.sample(replay_memory, batch_size)

def build_actor(state_input):
    actor_fc_1 = slim.fully_connected(state_input, hidden_size, activation_fn=tf.nn.relu)
    actor_fc_2 = slim.fully_connected(actor_fc_1, hidden_size, activation_fn=tf.nn.relu)
    actor_fc_3 = slim.fully_connected(actor_fc_2, hidden_size, activation_fn=tf.nn.relu)
    actor_fc_4 = slim.fully_connected(actor_fc_3, hidden_size, activation_fn=tf.nn.tanh)
    actor_output = slim.fully_connected(actor_fc_4, action_len, activation_fn=tf.nn.tanh) * 2
    return actor_output

def build_critic(state_input, action_input):
    critic_input = slim.flatten(tf.concat([state_input, action_input], axis=1))
    critic_fc_1 = slim.fully_connected(critic_input, hidden_size, activation_fn=tf.nn.relu)
    critic_fc_2 = slim.fully_connected(critic_fc_1, hidden_size, activation_fn=tf.nn.relu)
    critic_fc_3 = slim.fully_connected(critic_fc_2, hidden_size, activation_fn=tf.nn.tanh)
    critic_fc_4 = slim.fully_connected(critic_fc_3, hidden_size, activation_fn=tf.nn.tanh)
    critic_output = slim.fully_connected(critic_fc_4, 1, activation_fn=None)
    return critic_output

state_input_ph = tf.placeholder(tf.float32, shape=(None, ob_len))
action_input_ph = tf.placeholder(tf.float32, shape=(None, action_len))
target_q_ph = tf.placeholder(tf.float32, shape=(None, 1))

with tf.variable_scope("actor"):
    actor_output = build_actor(state_input_ph)

with tf.variable_scope("critic"):
    critic_output = build_critic(state_input_ph, action_input_ph)

with tf.variable_scope("target_actor"):
    target_actor_output = build_actor(state_input_ph)

with tf.variable_scope("target_critic"):
    target_critic_output = build_critic(state_input_ph, actor_output)

actor_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='actor')
critic_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='critic')
target_actor_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_actor')
target_critic_weights = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES, scope='target_critic')

update_target_ops = []
for i in range(len(actor_weights)):
    update_target_op = target_actor_weights[i].assign(tau*actor_weights[i] + (1-tau)*target_actor_weights[i])
    update_target_ops.append(update_target_op)
for i in range(len(critic_weights)):
    update_target_op = target_critic_weights[i].assign(tau*critic_weights[i] + (1-tau)*target_critic_weights[i])
    update_target_ops.append(update_target_op)

critic_lose = tf.reduce_mean(tf.square(target_q_ph-critic_output))
critic_optimizer = tf.train.AdamOptimizer(learn_rate).minimize(critic_lose, var_list=critic_weights)

actor_lose = tf.reduce_mean(-target_critic_output)
actor_optimizer = tf.train.AdamOptimizer(learn_rate).minimize(actor_lose, var_list=actor_weights)



def train():
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())

    def select_action(state):
        state = np.reshape(np.array(state), (-1, ob_len))
        action = sess.run(actor_output, {state_input_ph: state})
        return action

    def batch_updade(batch_size):
        batch = sample_from_memory(batch_size)
        state_0 = np.reshape(np.vstack([b[0] for b in batch]), (-1, ob_len))
        action_0 = np.reshape(np.vstack([b[1] for b in batch]), (-1, action_len))
        reward_0 = np.reshape(np.vstack([b[2] for b in batch]), (-1, 1))
        state_1 = np.reshape(np.vstack([b[3] for b in batch]), (-1, ob_len))
        action_1 = sess.run(actor_output, {state_input_ph:state_1})
        q = sess.run(critic_output, {state_input_ph:state_1, action_input_ph:action_1})
        target_q = reward_0 + discount_factor*q
        lose, _ = sess.run([critic_lose, critic_optimizer],
                             {state_input_ph:state_0, 
                              action_input_ph:action_0,
                              target_q_ph:target_q})

        lose, _ = sess.run([actor_lose, actor_optimizer],
                     {state_input_ph:state_0})

        sess.run(update_target_ops)


    env = gym.make(gym_env)
    env = gym.wrappers.Monitor(env, directory = 'pendulum', force=True)
    epoch = 3000

    def test():
        observation = env.reset()
        accreward = 0
        while True:
            env.render()
            action = select_action(observation)
            observation, reward, done, info = env.step(action)
            accreward += reward
            if done:
                print("test end with reward: {}".format(accreward))
                break

    noise_std = 4
    noise_rate = 0.995
    
    for ep in range(epoch):
        observation = env.reset()
        print("at ep: {}".format(ep))
        noise_std *= noise_rate
        cov1 = np.zeros((action_len, action_len), int)
        np.fill_diagonal(cov1, noise_std ** 2)
        while True:
            action = select_action(observation) + multivariate_normal(np.repeat(0, action_len), cov1)
            new_observation, reward, done, info = env.step(action)
            new_observation = np.reshape(new_observation, (-1, ob_len))
            replay_memory.append([observation, action, reward, new_observation])
            observation = new_observation
            if done:
                break

        for _ in range(min(100, len(replay_memory) // 256)):
            batch_updade(128)

        if (ep % 100 == 0):
            print("start test at ep: {}".format(ep))
            test()

In [5]:
train()

at ep: 0
start test at ep: 0
test end with reward: [-1623.58032227]
at ep: 1
at ep: 2
at ep: 3
at ep: 4
at ep: 5
at ep: 6
at ep: 7
at ep: 8
at ep: 9
at ep: 10
at ep: 11
at ep: 12
at ep: 13
at ep: 14
at ep: 15
at ep: 16
at ep: 17
at ep: 18
at ep: 19
at ep: 20
at ep: 21
at ep: 22
at ep: 23
at ep: 24
at ep: 25
at ep: 26
at ep: 27
at ep: 28
at ep: 29
at ep: 30
at ep: 31
at ep: 32
at ep: 33
at ep: 34
at ep: 35
at ep: 36
at ep: 37
at ep: 38
at ep: 39
at ep: 40
at ep: 41
at ep: 42
at ep: 43
at ep: 44
at ep: 45
at ep: 46
at ep: 47
at ep: 48
at ep: 49
at ep: 50
at ep: 51
at ep: 52
at ep: 53
at ep: 54
at ep: 55
at ep: 56
at ep: 57
at ep: 58
at ep: 59
at ep: 60
at ep: 61
at ep: 62
at ep: 63
at ep: 64
at ep: 65
at ep: 66
at ep: 67
at ep: 68
at ep: 69
at ep: 70
at ep: 71
at ep: 72
at ep: 73
at ep: 74
at ep: 75
at ep: 76
at ep: 77
at ep: 78
at ep: 79
at ep: 80
at ep: 81
at ep: 82
at ep: 83
at ep: 84
at ep: 85
at ep: 86
at ep: 87
at ep: 88
at ep: 89
at ep: 90
at ep: 91
at ep: 92
at ep: 93
at ep: 94
a

at ep: 712
at ep: 713
at ep: 714
at ep: 715
at ep: 716
at ep: 717
at ep: 718
at ep: 719
at ep: 720
at ep: 721
at ep: 722
at ep: 723
at ep: 724
at ep: 725
at ep: 726
at ep: 727
at ep: 728
at ep: 729
at ep: 730
at ep: 731
at ep: 732
at ep: 733
at ep: 734
at ep: 735
at ep: 736
at ep: 737
at ep: 738
at ep: 739
at ep: 740
at ep: 741
at ep: 742
at ep: 743
at ep: 744
at ep: 745
at ep: 746
at ep: 747
at ep: 748
at ep: 749
at ep: 750
at ep: 751
at ep: 752
at ep: 753
at ep: 754
at ep: 755
at ep: 756
at ep: 757
at ep: 758
at ep: 759
at ep: 760
at ep: 761
at ep: 762
at ep: 763
at ep: 764
at ep: 765
at ep: 766
at ep: 767
at ep: 768
at ep: 769
at ep: 770
at ep: 771
at ep: 772
at ep: 773
at ep: 774
at ep: 775
at ep: 776
at ep: 777
at ep: 778
at ep: 779
at ep: 780
at ep: 781
at ep: 782
at ep: 783
at ep: 784
at ep: 785
at ep: 786
at ep: 787
at ep: 788
at ep: 789
at ep: 790
at ep: 791
at ep: 792
at ep: 793
at ep: 794
at ep: 795
at ep: 796
at ep: 797
at ep: 798
at ep: 799
at ep: 800
start test at ep: 800

at ep: 1389
at ep: 1390
at ep: 1391
at ep: 1392
at ep: 1393
at ep: 1394
at ep: 1395
at ep: 1396
at ep: 1397
at ep: 1398
at ep: 1399
at ep: 1400
start test at ep: 1400
test end with reward: [-133.59855652]
at ep: 1401
at ep: 1402
at ep: 1403
at ep: 1404
at ep: 1405
at ep: 1406
at ep: 1407
at ep: 1408
at ep: 1409
at ep: 1410
at ep: 1411
at ep: 1412
at ep: 1413
at ep: 1414
at ep: 1415
at ep: 1416
at ep: 1417
at ep: 1418
at ep: 1419
at ep: 1420
at ep: 1421
at ep: 1422
at ep: 1423
at ep: 1424
at ep: 1425
at ep: 1426
at ep: 1427
at ep: 1428
at ep: 1429
at ep: 1430
at ep: 1431
at ep: 1432
at ep: 1433
at ep: 1434
at ep: 1435
at ep: 1436
at ep: 1437
at ep: 1438
at ep: 1439
at ep: 1440
at ep: 1441
at ep: 1442
at ep: 1443
at ep: 1444
at ep: 1445
at ep: 1446
at ep: 1447
at ep: 1448
at ep: 1449
at ep: 1450
at ep: 1451
at ep: 1452
at ep: 1453
at ep: 1454
at ep: 1455
at ep: 1456
at ep: 1457
at ep: 1458
at ep: 1459
at ep: 1460
at ep: 1461
at ep: 1462
at ep: 1463
at ep: 1464
at ep: 1465
at ep: 1466
at 

at ep: 2037
at ep: 2038
at ep: 2039
at ep: 2040
at ep: 2041
at ep: 2042
at ep: 2043
at ep: 2044
at ep: 2045
at ep: 2046
at ep: 2047
at ep: 2048
at ep: 2049
at ep: 2050
at ep: 2051
at ep: 2052
at ep: 2053
at ep: 2054
at ep: 2055
at ep: 2056
at ep: 2057
at ep: 2058
at ep: 2059
at ep: 2060
at ep: 2061
at ep: 2062
at ep: 2063
at ep: 2064
at ep: 2065
at ep: 2066
at ep: 2067
at ep: 2068
at ep: 2069
at ep: 2070
at ep: 2071
at ep: 2072
at ep: 2073
at ep: 2074
at ep: 2075
at ep: 2076
at ep: 2077
at ep: 2078
at ep: 2079
at ep: 2080
at ep: 2081
at ep: 2082
at ep: 2083
at ep: 2084
at ep: 2085
at ep: 2086
at ep: 2087
at ep: 2088
at ep: 2089
at ep: 2090
at ep: 2091
at ep: 2092
at ep: 2093
at ep: 2094
at ep: 2095
at ep: 2096
at ep: 2097
at ep: 2098
at ep: 2099
at ep: 2100
start test at ep: 2100
test end with reward: [-135.54673767]
at ep: 2101
at ep: 2102
at ep: 2103
at ep: 2104
at ep: 2105
at ep: 2106
at ep: 2107
at ep: 2108
at ep: 2109
at ep: 2110
at ep: 2111
at ep: 2112
at ep: 2113
at ep: 2114
at 

at ep: 2690
at ep: 2691
at ep: 2692
at ep: 2693
at ep: 2694
at ep: 2695
at ep: 2696
at ep: 2697
at ep: 2698
at ep: 2699
at ep: 2700
start test at ep: 2700
test end with reward: [-384.47573853]
at ep: 2701
at ep: 2702
at ep: 2703
at ep: 2704
at ep: 2705
at ep: 2706
at ep: 2707
at ep: 2708
at ep: 2709
at ep: 2710
at ep: 2711
at ep: 2712
at ep: 2713
at ep: 2714
at ep: 2715
at ep: 2716
at ep: 2717
at ep: 2718
at ep: 2719
at ep: 2720
at ep: 2721
at ep: 2722
at ep: 2723
at ep: 2724
at ep: 2725
at ep: 2726
at ep: 2727
at ep: 2728
at ep: 2729
at ep: 2730
at ep: 2731
at ep: 2732
at ep: 2733
at ep: 2734
at ep: 2735
at ep: 2736
at ep: 2737
at ep: 2738
at ep: 2739
at ep: 2740
at ep: 2741
at ep: 2742
at ep: 2743
at ep: 2744
at ep: 2745
at ep: 2746
at ep: 2747
at ep: 2748
at ep: 2749
at ep: 2750
at ep: 2751
at ep: 2752
at ep: 2753
at ep: 2754
at ep: 2755
at ep: 2756
at ep: 2757
at ep: 2758
at ep: 2759
at ep: 2760
at ep: 2761
at ep: 2762
at ep: 2763
at ep: 2764
at ep: 2765
at ep: 2766
at ep: 2767
at 